<a href="https://colab.research.google.com/github/WayneGretzky1/CSCI-4521-Applied-Machine-Learning/blob/main/zzzvold_1_3_kNN_Iris_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/be-prado/csci4521/refs/heads/main/iris.csv")
names = list(df.columns) #Save column names


Displaying the dataset.

In [2]:
df

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [8]:
#Choose which data is featuers (X) and labels (y)
X = df[ ['sepal.length', 'sepal.width','petal.length','petal.width'] ].to_numpy()
y = df['variety'].to_numpy()

for a in y['variety']:
  if a == "Setosa":
    a = 0
  if a == "Vericolor":
    a = 1
  if a == "Virginia":
    a = 2
y.head()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [4]:
def train_test_split(X,y,test_percent):
  numRows = y.size                     #number or rows in the entire dataset
  splitPoint = int((1-test_percent)*numRows)  #the row index test_size% of the way through
  p = np.random.permutation(numRows)   #array for shuffling data
  X = X[p]
  y = y[p]

  #Training data array
  X_train = X[:splitPoint]  # training features
  y_train = y[:splitPoint]  # training labels

  #Testing data array
  X_test = X[splitPoint:]  # testing features
  y_test = y[splitPoint:] # testing labels

  return X_train, X_test, y_train, y_test

In [5]:
def nn_classify_sample(train_features, train_labels, new_feature):
  dists = np.array([np.sum( (t-new_feature)**2) for t in train_features])
  nearest = dists.argmin()
  return train_labels[nearest]

def nn_classify(train_features, train_labels, test_features):
  num_result = test_features[:,0].size
  result = np.ones(num_result)
  for i in range(0,num_result):
    result[i] = nn_classify_sample(train_features[:,0:2], train_labels, test_features[i])
  return result

In [6]:
def accuracy(classify, X_train, y_train, X_test, y_test):
  pred = classify(X_train, y_train, X_test)
  correct_pred = (pred == y_test)
  total_preds = y_test.size
  return correct_pred.sum()/total_preds

In [7]:
def avg_accuracy(classifier, X, y, count = 200, split=0.33):
  acc = 0
  for _ in range(0,count):
    X_train, X_test, y_train, y_test = train_test_split(X,y,split)
    acc += accuracy(classifier, X_train, y_train, X_test, y_test)
  return acc/count

In [ ]:
# TODO: create a train/test split where 1/3 of the data is used for testing
x_train, x_test, y_train, y_test = train_test_split(X, y, 0.33)
print(X.shape)
print(x_train.shape)
print(x_test.shape)
accuracy(nn_classify, x_train, y_train, x_test, y_test)


In [ ]:
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# TODO: write a function that will use KNN to classify a whole array of test features
# Hint: use a wrapper function so that it fits within our accuracy helper function!
def knn_classifier(k):
    def knn_classify(x_train, y_train, x_test):
        knn = KNeighborsClassifier(n_neighbors = k)
        knn.fit(x_train, y_train)

        return knn.predict(x_test)
    return knn_classify

In [ ]:
# TODO: test it out with 2, 5, and 10 neighbors
x_train, x_test, y_train, y_test = train_test_split(X, y, 0.33)
knn = knn_classifier(10)
accuracy(knn, x_train, y_train, x_test, y_test)


In [ ]:
# TODO: find the best performing K testing every third value between 1 - 90
ks = np.arange(1, 51, 2)
test_acc_list = []
train_acc_list = []

def accuracy(classify, X_train, y_train, X_test, y_test):
  pred = classify(X_train, y_train, X_test)
  correct_pred = (pred == y_test)
  total_preds = y_test.size
  return correct_pred.sum()/total_preds

for k in ks:
  my_knn = knn_classifier(k)
  test_acc = 0
  train_acc = 0
  for i in range(20): # avg over 20 train-test splits
    X_train, y_train, X_test, y_test = train_test_split(X, y, 0.33)
    test_acc += accuracy(my_knn, X_train, y_train, X_test, y_test) / 20
    train_acc += accuracy(my_knn, X_train, y_train, X_train, y_train) / 20
  train_acc_list.append(train_acc)
  test_acc_list.append(train_acc)
